In [52]:
os.chdir(r'C:\Users\bgamb\PycharmProjects\IB\Data')

NameError: name 'os' is not defined

In [51]:
import numpy as np
import keras
import pandas as pd
from tensorflow.keras.layers import Dense
from keras.src.optimizers import Adam
from collections import deque
import random
import pickle

In [1]:
from keras.models import load_model

In [34]:
from datetime import datetime
from sklearn.cluster import KMeans
import os, glob, sys
import time, threading
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [7]:
model = keras.models.Sequential()
model.add(keras.Input((8,)))
model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [122]:
model = load_model('dqn_trading_model_400.keras')

In [28]:
def bollinger_bands(series: pd.Series, length: int = 20, *, num_stds: tuple[float, ...] = (2, 0, -2), prefix: str = '') -> pd.DataFrame:
    # Ref: https://stackoverflow.com/a/74283044/
    rolling = series.rolling(length)
    bband0 = rolling.mean()
    bband_std = rolling.std(ddof=0)
    return pd.DataFrame({f'{prefix}{num_std}': (bband0 + (bband_std * num_std)) for num_std in num_stds})

In [30]:
def agregate_indicator(data, column, indicator, length = 20, num_stds = (2, 0, -2), prefix='BB_'):
    if indicator == 'bollinger':
        b_bands = bollinger_bands(data[column], length, num_stds=num_stds, prefix=prefix)
    data = data.join(b_bands, on=['DateTime']).iloc[length:]
    return data

In [29]:
def agregate_categories(data, n_clusters):
    # Convert DataFrame to matrix
    mat = data.values
    # Using sklearn
    km = KMeans(n_clusters=n_clusters)
    km.fit(mat)
    # Get cluster assignment labels
    labels = km.labels_
    # Format results as a DataFrame
    results = pd.DataFrame([data.index, labels]).T
    results.columns = ['DateTime', 'Category']
    results.set_index('DateTime', inplace=True)
    results = pd.to_numeric(results['Category'])
    data = data.join(results, on=['DateTime'])
    return data

In [26]:
ticker = 'BTC'
path = os.path.join(ticker, '*')
path_to_file = glob.glob(path)[-1]

In [35]:
data = pd.read_csv(path_to_file, index_col='DateTime')
data = agregate_indicator(data, column='Close', indicator='bollinger',
                          length=10,num_stds=(1.5, 0, -1.5),
                          prefix='CloseBB5_')
test_data = agregate_categories(data, n_clusters=25)

C:\Users\bgamb\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\bgamb\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


In [36]:
test_data.head()

,Open,High,Low,Close,CloseBB5_1.5,CloseBB5_0,CloseBB5_-1.5,Category
DateTime,,,,,,,,
2024-04-03 06:00:00,66127.00,66417.50,66104.25,66406.75,66431.955905,65930.450,65428.944095,13
2024-04-03 07:00:00,66406.75,66509.00,66180.75,66187.75,66478.527171,65978.375,65478.222829,13
2024-04-03 08:00:00,66187.50,66664.75,66065.00,66550.25,66604.583645,66050.475,65496.366355,13
2024-04-03 09:00:00,66550.00,66648.00,66400.00,66514.00,66658.491042,66155.550,65652.608958,13
2024-04-03 10:00:00,66513.75,66551.50,66090.00,66216.25,66572.129268,66236.575,65901.020732,13


In [48]:
x = np.array(test_data.iloc[20].to_list())
model.predict(x.reshape(1,8))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step


array([[-46509.055]], dtype=float32)

In [53]:
np.load('q_table400.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'q_table400.npy'

In [1]:
with open(f"q_memory300", "rb") as reader:
    memory = pickle.load(reader)
reader.close()

FileNotFoundError: [Errno 2] No such file or directory: 'q_memory300'

In [132]:
states = [inner[:-1] for inner in memory]
actions = [inner[-1] for inner in memory]


In [2]:
action_array = [[1 if i==action else 0 for i in range(0,3)] for action in actions]

NameError: name 'actions' is not defined

In [138]:
X_train = np.array(states[:round(len(X)*0.8)])
Y_train = np.array(action_array[:round(len(Y)*0.8)])
X_test = np.array(states[round(len(X)*0.8):])
Y_test = np.array(action_array[round(len(X)*0.8):])

In [169]:
model = keras.models.Sequential()
model.add(keras.Input((8,)))
model.add(Dense(3, activation='relu', name='hidden_layer_1'))
model.add(Dense(4, activation='relu', name='hidden_layer_2'))
model.add(Dense(3, activation='sigmoid', name='hidden_layer_3'))
model.add(Dense(3, activation='softmax', name='output_layer'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.15), metrics=['accuracy'])
print('Model ready')

Model ready


In [170]:
start = time.time()
model.fit(X_train, Y_train, batch_size=30, epochs=10)
end = time.time()
(end-start)/60

Epoch 1/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 75s 6ms/step - accuracy: 0.3514 - loss: 1.1224
Epoch 2/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 66s 6ms/step - accuracy: 0.3522 - loss: 1.1214
Epoch 3/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 61s 6ms/step - accuracy: 0.3533 - loss: 1.1202
Epoch 4/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 53s 5ms/step - accuracy: 0.3511 - loss: 1.1222
Epoch 5/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 61s 6ms/step - accuracy: 0.3513 - loss: 1.1234
Epoch 6/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 67s 6ms/step - accuracy: 0.3519 - loss: 1.1215
Epoch 7/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.3530 - loss: 1.1216
Epoch 8/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 73s 7ms/step - accuracy: 0.3513 - loss: 1.1233
Epoch 9/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 67s 6ms/step - accuracy: 0.3530 - loss: 1.1216
Epoch 10/10
10523/10523 ━━━━━━━━━━━━━━━━━━━━ 52s 5ms/step - accuracy: 0.3488 - loss: 1.1228


10.950359825293223

In [166]:
score = model.evaluate(X_test, Y_test)

2467/2467 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.4078 - loss: 9.5450


In [149]:
score

#X_test[2].reshape(1,8)

[1.0837546586990356, 0.40918880701065063]

In [167]:
model.predict(np.array(X_test[80]).reshape(1,8))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step


array([[0.      , 0.      , 6.471728]], dtype=float32)

In [168]:
Y_test[80]

array([1, 0, 0])